# Ethical Reminder (from the Lab Alliance Compact)

- Data belongs to truth, not expectations; document analysis transparently.  
- Perform **your own work**; cite all sources properly.  
- Respect collaboration and uphold psychological safety.  

> By proceeding, you acknowledge the Compact and agree to act accordingly.

# Gaussian Fit Pilot

**Learning goals**
1. Generate synthetic Gaussian data with known ground truth.
2. Perform nonlinear fit using `scipy.optimize.curve_fit`.
3. Interpret parameter uncertainties.
4. Diagnose fits with residuals.
5. Understand systematic vs. statistical uncertainty in peak fitting.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
rng = np.random.default_rng(123)

def gauss(x, A, x0, sigma, C):
    return A * np.exp(-(x-x0)**2/(2*sigma**2)) + C

# True parameters
A_true, x0_true, sigma_true, C_true = 5.0, 0.0, 1.0, 0.5
N = 80
x = np.linspace(-5, 5, N)
y = gauss(x, A_true, x0_true, sigma_true, C_true) + rng.normal(0, 0.2, size=N)

plt.plot(x, y, 'o', label='data')
plt.xlabel('x'); plt.ylabel('y')
plt.title('Synthetic Gaussian Data')
plt.legend(); plt.show()


## Perform Gaussian fit
We use `curve_fit` to estimate A, x0, σ, C.

In [ ]:
p0 = [4.0, 0.5, 0.8, 0.0]
popt, pcov = curve_fit(gauss, x, y, p0=p0)
perr = np.sqrt(np.diag(pcov))
for name, val, err, tru in zip(['A','x0','sigma','C'], popt, perr,
                               [A_true,x0_true,sigma_true,C_true]):
    print(f"{name:5s} = {val:7.3f} ± {err:5.3f} (true {tru:5.3f})")


In [ ]:
y_fit = gauss(x, *popt)
res = y - y_fit

fig, (ax1, ax2) = plt.subplots(2,1,figsize=(6,6),sharex=True)
ax1.plot(x,y,'o',label='data')
ax1.plot(x,y_fit,'r-',label='fit')
ax1.legend(); ax1.set_ylabel('y')

ax2.axhline(0,color='k',lw=1)
ax2.plot(x,res,'o')
ax2.set_xlabel('x'); ax2.set_ylabel('residuals')
ax2.set_title('Residuals')
plt.tight_layout(); plt.show()


## Exercises
1. Add random outliers to the data. How robust is the Gaussian fit?
2. Change initial guesses `p0` — does the fit still converge?
3. Increase noise variance. How do uncertainties in parameters scale?
4. Introduce a systematic shift (add +0.1 to all y-values). Which parameter absorbs it? Why?
